<a href="https://colab.research.google.com/github/NattyPavez/Agente-Gemini-CV-conversor-portafolio/blob/main/proyecto_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto paralelo a curso Inmersión AI de Alura con Google Gemini

Primera parte

Librerías utilizadas

In [ ]:
!pip install -q langchain-google-genai google-generativeai

In [ ]:
!pip install email-validator

In [ ]:
#permite leer PDF
!pip install pymupdf

In [ ]:
# permite crear una interfaz simple con muy pocas líneas de código
!pip install -q gradio

Importación api key de AIStudio

In [ ]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('personal_gemini_api_key')

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
# Importa la clase de incrustaciones
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import userdata

#  modelo de incrustaciones para la clasificación de perfiles
embeddings_model = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=GOOGLE_API_KEY
    )

### Asistente de recuperacion de datos de Curriculum Vitae para generar portafolio profesional

In [ ]:
from pydantic import BaseModel, Field, EmailStr
from typing import List, Optional

class Habilidades(BaseModel):
    tecnicas: List[str] = Field(default_factory=list)
    blandas: List[str] = Field(default_factory=list)

class ExperienciaLaboral(BaseModel):
    empresa_o_proyecto: str
    cargo: str
    periodo: str
    descripcion: str

class Educacion(BaseModel):
    institucion: str
    titulo: str
    periodo: str

class Perfil(BaseModel):
    nombre: str
    email: EmailStr
    telefono: Optional[str] = None
    ciudad_pais: str

class CurriculumVitae(BaseModel):
    perfil: Perfil
    resumen: str
    experiencia_laboral: List[ExperienciaLaboral] = Field(default_factory=list)
    educacion: List[Educacion] = Field(default_factory=list)
    habilidades: Habilidades
    puntos_clave_sugeridos: List[str] = Field(default_factory=list)



In [ ]:
PROMPT_BASE_EXTRACCION = """Eres un asistente experto en análisis de currículums. Tu tarea es extraer la información clave de un currículum vitae y estructurarla en un objeto JSON.

La información que debes extraer es:
1. **Información personal**: Nombre completo, email, teléfono, ciudad y país.
2. **Resumen profesional**: Un breve resumen que destaque las principales habilidades y experiencia.
3. **Experiencia laboral**: Para cada puesto, extrae el nombre de la empresa, cargo, período (ej: "2018 - 2022") y una descripción breve de las responsabilidades y logros clave.
4. **Educación**: Para cada título, extrae el nombre de la institución, el título obtenido y el período (ej: "2015 - 2018").
5. **Habilidades**: Lista de habilidades técnicas (ej: "Python", "SQL", "Git") y habilidades blandas (ej: "Liderazgo", "Comunicación").

Después de extraer la información, evalúa la profesión del currículum y sugiere qué información adicional sería valiosa para destacar en un portafolio web. Estas sugerencias deben basarse en estándares de la industria para esa profesión. Devuelve esta sugerencia en un campo llamado "puntos_clave_sugeridos".

Si el perfil detectado es '{perfil_detectado}', asegúrate de adaptar la extracción de la siguiente manera:
{instruccion_especifica}
"""

Clasificación de perfiles utilizando Emmbeddins

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from typing import Dict
from langchain_core.messages import SystemMessage, HumanMessage

# Perfiles de referencia
perfiles_referencia = {
    "Perfil Administrativo / Asistente": "Profesional con experiencia en gestión de agendas, coordinación de equipos y atención al cliente. Destacada capacidad organizativa, manejo de herramientas ofimáticas (Excel, Word, Power Point) y optimización de procesos administrativos.",
    "Perfil Comercial / Ventas": "Ejecutivo de ventas con trayectoria en el cumplimiento de metas comerciales, negociación, cierre de tratos y fidelización de clientes. Orientado a resultados y con fuerte habilidad para generar nuevas oportunidades de negocio, prospección y gestión de cartera.",
    "Perfil Tecnológico / TI": "Desarrollador de software con experiencia en lenguajes de programación como Java, Python, JavaScript. Manejo de frameworks como Spring Boot, Django, o React. Experto en bases de datos SQL y NoSQL, desarrollo de APIs y metodologías ágiles (Scrum, Kanban).",
    "Perfil Marketing / Comunicación": "Especialista en marketing digital con dominio de campañas en redes sociales (Facebook Ads, Google Ads), SEO, SEM y analítica web. Capacidad para crear estrategias de contenido que aumentan la visibilidad y el engagement de marcas, email marketing y relaciones públicas.",
    "Perfil Financiero / Contabilidad": "Contador auditor con sólida experiencia en análisis financiero, control de gastos, auditoría, planificación tributaria y cumplimiento normativo. Fuerte orientación a la precisión, la eficiencia en procesos contables y la elaboración de reportes financieros.",
    "Perfil de Salud": "Profesional de la salud con experiencia en atención clínica, diagnóstico, tratamiento y enfoque en la calidad del servicio al paciente. Capacitado en protocolos de emergencia, prevención de enfermedades, promoción de la salud y educación a la comunidad.",
    "Perfil de Recursos Humanos": "Especialista en gestión de talento, con experiencia en reclutamiento y selección, capacitación, evaluación de desempeño y clima organizacional. Enfocado en potenciar equipos de alto desempeño y en la aplicación de políticas de bienestar laboral.",
    "Perfil de Educación / Docencia": "Docente con experiencia en la enseñanza de asignaturas como [ej: matemáticas, historia] en [ej: nivel escolar o universitario]. Capaz de diseñar planes pedagógicos efectivos, aplicar metodologías innovadoras y adaptarse a distintos estilos de aprendizaje. Enfocado en la motivación y el desarrollo integral de los estudiantes.",
    "Perfil de Logística / Operaciones": "Profesional en logística con trayectoria en gestión de inventarios, control de bodegas, cadena de suministro y optimización de procesos de distribución. Orientado a la eficiencia operativa, reducción de costos, cumplimiento de tiempos de entrega y manejo de sistemas ERP.",
    "Perfil de Atención al Cliente / Servicios": "Especialista en atención al cliente con experiencia en resolución de problemas, manejo de reclamos y generación de experiencias positivas. Destacado por la empatía, comunicación efectiva, orientación a la satisfacción del usuario y manejo de sistemas CRM."
}

# incrustaciones una sola vez al inicio del programa
vectores_referencia = {
    perfil: embeddings_model.embed_query(texto)
    for perfil, texto in perfiles_referencia.items()
}

def clasificar_perfil(texto_cv: str) -> str:
    """Clasifica el perfil de un CV comparando su incrustación con perfiles de referencia."""
    if not texto_cv:
        return "Perfil no definido"
    try:
        vector_cv = embeddings_model.embed_query(texto_cv)
        mejor_coincidencia = "No se pudo clasificar"
        mayor_similitud = -1
        for perfil, vector_ref in vectores_referencia.items():
            similitud = cosine_similarity(np.array(vector_cv).reshape(1, -1), np.array(vector_ref).reshape(1, -1))[0][0]
            if similitud > mayor_similitud:
                mayor_similitud = similitud
                mejor_coincidencia = perfil
        print(f"El sistema ha clasificado este perfil como: {mejor_coincidencia} (Similitud: {mayor_similitud:.2f})")
        return mejor_coincidencia
    except Exception as e:
        print(f"Ocurrió un error en la clasificación: {e}")
        return "Error en la clasificación"

llm_cv_chain = llm.with_structured_output(CurriculumVitae)

def extraer_y_clasificar_cv(texto_cv: str):
    """
    Función principal que clasifica el CV y extrae los datos con un prompt dinámico.
    """
    perfil_clasificado = clasificar_perfil(texto_cv)
    print(f"Perfil detectado: {perfil_clasificado}")

    if "Tecnológico" in perfil_clasificado or "freelance" in texto_cv.lower():
        instruccion = "Para la experiencia laboral, si se menciona un proyecto o cliente, usa el nombre del proyecto en lugar del nombre de la empresa. En las habilidades, prioriza los lenguajes de programación, frameworks y herramientas tecnológicas."
    else:
        instruccion = "Sigue el formato estándar de experiencia laboral, usando el nombre de la empresa en cada puesto."

    prompt_dinamico = PROMPT_BASE_EXTRACCION.format(perfil_detectado=perfil_clasificado, instruccion_especifica=instruccion)

    salida = llm_cv_chain.invoke([
        SystemMessage(content=prompt_dinamico),
        HumanMessage(content=texto_cv)
    ])

    return salida

Permitir carga de CV por interfaz

In [ ]:
import gradio as gr
from typing import IO
import fitz
import json

def leer_pdf(ruta_del_archivo):
    """
    Lee un archivo PDF y extrae todo su texto en una sola cadena.
    """
    try:
        documento = fitz.open(ruta_del_archivo)
        texto_completo = ""
        for pagina in documento:
            texto_completo += pagina.get_text()
        return texto_completo
    except Exception as e:
        print(f"Error al leer el PDF: {e}")
        return ""

# Modificamos la función para que retorne la clasificación y el JSON
def extraer_y_clasificar_cv(texto_cv: str):
    perfil_clasificado = clasificar_perfil(texto_cv)
    print(f"Perfil detectado: {perfil_clasificado}")

    if "Tecnológico" in perfil_clasificado or "freelance" in texto_cv.lower():
        instruccion = "Para la experiencia laboral, si se menciona un proyecto o cliente, usa el nombre del proyecto en lugar del nombre de la empresa. En las habilidades, prioriza los lenguajes de programación, frameworks y herramientas tecnológicas."
    else:
        instruccion = "Sigue el formato estándar de experiencia laboral, usando el nombre de la empresa en cada puesto."

    prompt_dinamico = PROMPT_BASE_EXTRACCION.format(
        perfil_detectado=perfil_clasificado,
        instruccion_especifica=instruccion
    )

    salida = llm_cv_chain.invoke([
        SystemMessage(content=prompt_dinamico),
        HumanMessage(content=texto_cv)
    ])

    # Convertir a objeto CurriculumVitae válido
    if isinstance(salida, dict):
        datos_cv = CurriculumVitae.model_validate(salida)
    else:
        datos_cv = salida  # ya es un CurriculumVitae

    return perfil_clasificado, datos_cv


def procesar_cv_desde_interfaz(archivo_pdf: IO):
    """
    Función que recibe un archivo PDF desde la interfaz, lo lee y lo procesa.
    """
    if archivo_pdf is None:
        return "Por favor, sube un archivo PDF.", ""  # Retorna dos valores
    try:
        texto_del_cv = leer_pdf(archivo_pdf.name)

        if not texto_del_cv:
            return "El archivo PDF no contiene texto extraíble.", "" # Retorna dos valores

        # Llamas a la función principal que ahora devuelve dos valores
        perfil_detectado, datos_extraidos = extraer_y_clasificar_cv(texto_del_cv)

        # Retornamos los dos valores esperados por Gradio
        return perfil_detectado, datos_extraidos.model_dump_json(indent=2)
    except Exception as e:
        return f"Ocurrió un error al procesar el archivo: {e}", ""

# Creamos la interfaz de Gradio con múltiples salidas
interfaz = gr.Interface(
    fn=procesar_cv_desde_interfaz,
    inputs=gr.File(type="filepath", label="Sube tu CV en formato PDF"),
    outputs=[
        gr.Textbox(label="Clasificación del perfil"),
        gr.Json(label="Datos del CV extraídos")
    ],
    title="Asistente para Creación de Portafolios",
    description="Sube tu currículum vitae en PDF y obtén los datos estructurados en formato JSON."
)

# Lanzamos la interfaz.
interfaz.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2ee5d8f4fc8970850b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Segunda parte

In [ ]:
# Prompt mejorado para generación de HTML editable
PROMPT_GENERADOR_HTML_EDITABLE = """
Eres un diseñador y desarrollador frontend senior con excelente criterio UX/UI.
Tu tarea es generar un ÚNICO archivo HTML autocontenido (<!doctype html> …) que sirva como **portafolio personal editable**.

🚨 INSTRUCCIONES TÉCNICAS:
- Devuelve SOLO el código HTML completo, sin explicaciones fuera del <html>.
- El HTML debe ser semántico y accesible (usar <header>, <nav>, <main>, <section>, <footer>).
- Incluye en el <head>:
  - <meta name="viewport" content="width=device-width, initial-scale=1.0">
  - Google Fonts (ej: Inter, Lato, o Roboto).
  - <style> con todas las variables CSS para colores, tipografía y espaciados (no uses archivos externos).
- Incrusta el JSON original EXACTAMENTE como:
  <script type="application/json" id="cv-data">{datos_cv_json}</script>
- Marca elementos editables con:
  - data-editable="true"
  - data-field="ruta.en.el.json"  (ej: data-field="perfil.nombre")

🎨 ESTILO Y PALETA SEGÚN PERFIL:
- Perfil Tecnológico/TI → tema oscuro, acentos azules/cian, tipografía monoespaciada para títulos de proyectos.
- Perfil Salud/Educación → tema claro, verdes/azules suaves, tipografía clara, enfoque humano.
- Perfil Marketing/Comercial → colores vibrantes (naranja, rojo, púrpura), secciones dinámicas con métricas.
- Perfil Financiero/Administrativo → diseño sobrio, paleta azul marino/gris/blanco, tipografía seria.
- Otros perfiles → paleta neutra (gris claro/fondo blanco, texto oscuro).
- Siempre mobile-first y accesible (contraste adecuado, botones grandes, aria-labels).

📌 ESTRUCTURA DEL PORTAFOLIO:
HEADER (sticky/top):
- Siempre visible, con nombre + botones de navegación (Perfil, Sobre mí, Experiencia, Contacto).
- Toggle idioma (ES/EN).
- Scroll suave al hacer clic en botones.

1) PERFIL:
- Foto circular (placeholder si no hay), nombre grande, ciudad/país, breve presentación (2-3 líneas).
- data-fields por cada dato.

2) SOBRE MÍ:
- Resumen profesional redactado a partir del JSON.
- Bloques con EDUCACIÓN, HABILIDADES TÉCNICAS y HABILIDADES BLANDAS.
- IA puede enriquecer el texto si faltan datos → redactar de forma profesional.

3) EXPERIENCIA / PROYECTOS:
- Grid responsive (3 columnas en desktop, 1 en móvil).
- Cada tarjeta con:
  - Imagen (placeholder si no hay)
  - Cargo, empresa/proyecto, periodo
  - Breve descripción
  - Enlaces opcionales (GitHub, demo, etc.)
- Renderizar hasta 5 experiencias visibles; si faltan datos, mostrar placeholders claros (“Agregar experiencia”).

4) CONTACTO:
- Email (con botón “copiar”).
- Redes sociales: GitHub, LinkedIn, Twitter, web personal (si faltan → placeholders).
- Formulario simple (nombre, email, mensaje).

FOOTER:
- Texto de derechos reservados.
- “Creado con IA y HTML” + link al repositorio o placeholder.
- Mantener diseño consistente.

⚡️ FUNCIONALIDAD EXTRA:
- Toolbar flotante con botones:
  - Editar (activa/desactiva contentEditable en elementos con data-editable).
  - Descargar HTML.
  - Descargar JSON (#cv-data).
  - Cambiar idioma ES/EN.
- Scroll suave en navegación interna.
- Sombras sutiles en tarjetas, hover animado.

⚠️ MUY IMPORTANTE:
- Si faltan datos en el JSON, coloca placeholders claros (“Agregar foto”, “Agregar descripción”).
- Ocupa bien el espacio en cualquier dispositivo (layout responsivo).
- Fondo oscuro + texto claro por defecto, salvo perfiles Salud/Educación (fondo claro).
- Devuelve SOLO el HTML válido, nada más.

JSON que debes usar para poblar el contenido:
{datos_cv_json}

Perfil detectado (adapta la paleta y estilo según este): {perfil_detectado}
"""


In [ ]:
# Función que llama a Gemini y devuelve HTML editable
from langchain_core.messages import SystemMessage, HumanMessage
import json
import html as html_lib

def _strip_code_fences(text: str) -> str:
    if not text:
        return ""
    t = text.strip()
    if t.startswith("```"):
        # quitar fence de apertura
        # soporte para ```html o ```
        parts = t.split("\n")
        # si la primera linea es fence, lo quitamos
        if parts and parts[0].startswith("```"):
            parts = parts[1:]
        # si la última linea es ```
        if parts and parts[-1].strip().startswith("```"):
            parts = parts[:-1]
        t = "\n".join(parts)
    return t.strip()

def generar_portfolio_html_editable(datos_cv, perfil_clasificado: str) -> str:
    """
    Genera HTML editable a partir de datos_cv (puede ser Pydantic model o dict)
    Usa PROMPT_GENERADOR_HTML_EDITABLE y llm_generador (o llm como fallback).
    """
    # convierte a dict si el input es Pydantic
    if hasattr(datos_cv, "model_dump"):
        datos_cv_dict = datos_cv.model_dump()
    elif isinstance(datos_cv, dict):
        datos_cv_dict = datos_cv
    else:
        # si vino como string JSON
        try:
            datos_cv_dict = json.loads(datos_cv)
        except Exception:
            datos_cv_dict = {}

    datos_cv_json_str = json.dumps(datos_cv_dict, ensure_ascii=False, indent=2)

    prompt_final = PROMPT_GENERADOR_HTML_EDITABLE.format(
        datos_cv_json=datos_cv_json_str,
        perfil_detectado=perfil_clasificado
    )

    # eligir cliente LLM: primero llm_generador (Jules), luego llm (tu inicial)
    model_client = globals().get("llm_generador") or globals().get("llm")
    if model_client is None:
        raise RuntimeError("No se encontró `llm_generador` ni `llm`. Define tu cliente Gemini antes de ejecutar esta celda.")

    try:
        print("Invocando al modelo generador (HTML editable)...")
        response = model_client.invoke([
            SystemMessage(content="Eres un experto en diseño web y accesibilidad. Responde ÚNICAMENTE con el HTML solicitado."),
            HumanMessage(content=prompt_final)
        ])
       # Extraer texto de la respuesta (varios SDKs usan .content o .text)
        html_raw = ""
        if hasattr(response, "content"):
            html_raw = response.content
        elif hasattr(response, "text"):
            html_raw = response.text
        else:
            html_raw = str(response)

        html_raw = _strip_code_fences(html_raw)

    # Asegurar que el JSON original esté embebido como <script id="cv-data"> ... </script>
        if '<script type="application/json" id="cv-data">' not in html_raw:
            injection = f'\n<script type="application/json" id="cv-data">{html_lib.escape(datos_cv_json_str)}</script>\n'
            lower = html_raw.lower()
            if "<body" in lower:
                idx = lower.find("<body")
                insert_at = html_raw.find(">", idx) + 1
                html_raw = html_raw[:insert_at] + injection + html_raw[insert_at:]
            else:
                # si no hay body, envolver todo
                html_raw = "<!doctype html>\n<html lang='es'>\n<head><meta charset='utf-8'></head>\n<body>" + injection + html_raw + "</body></html>"

        # Devolver HTML final (listo para guardar)
        print("HTML editable generado correctamente.")
        return html_raw

    except Exception as e:
        print(f"Error durante la generación del HTML: {e}")
        return f"<html><body><h1>Error al generar el portafolio</h1><p>{e}</p></body></html>"

In [ ]:
# Inserta una toolbar editable si el HTML no la trae
def ensure_editable_toolbar(html: str) -> str:
    """
    Si el HTML ya trae una toolbar con id="editable-toolbar", lo deja tal cual.
    Si no, inserta una toolbar ligera (botones: Editar / Descargar HTML / Descargar JSON).
    """
    if html is None:
        return html or "<html><body></body></html>"

    if 'id="editable-toolbar"' in html:
        return html  # ya la trae el modelo

    toolbar_html = """
    <!-- EDITABLE_TOOLBAR: Minimal toolbar para edición rápida. El frontend dev puede reemplazarla. -->
    <div id="editable-toolbar" style="position:fixed;right:12px;top:12px;z-index:2147483647;padding:8px;border-radius:8px;box-shadow:0 2px 8px rgba(0,0,0,0.12);background:rgba(255,255,255,0.95);font-family:Arial,Helvetica,sans-serif;">
      <button onclick="(function(){var es=document.querySelectorAll('[data-editable]'); es.forEach(e=>e.contentEditable = e.isContentEditable? 'false':'true'); this.textContent = (this.textContent==='Editar')? 'Salir':'Editar';}).call(this)">Editar</button>
      <button onclick="(function(){var a=document.createElement('a'); var html='<!doctype html>'+document.documentElement.outerHTML; a.href='data:text/html;charset=utf-8,'+encodeURIComponent(html); a.download='portafolio_editado.html'; a.click(); })()">Descargar HTML</button>
      <button onclick="(function(){var d=document.getElementById('cv-data'); if(!d) return alert('No hay JSON embebido'); var a=document.createElement('a'); a.href='data:application/json;charset=utf-8,'+encodeURIComponent(d.textContent); a.download='cv_datos.json'; a.click(); })()">Descargar JSON</button>
      <div style="font-size:11px;color:#333;margin-top:6px;">(Los elementos editables tienen <code>data-editable=\"true\"</code>)</div>
    </div>
    """

    # insertar después de <body> si existe
    lower = html.lower()
    if "<body" in lower:
        idx = lower.find("<body")
        insert_at = html.find(">", idx) + 1
        html = html[:insert_at] + toolbar_html + html[insert_at:]
    else:
        # si no hay <body>, envolver todo con estructura mínima
        html = "<!doctype html><html><head><meta charset='utf-8'></head><body>" + toolbar_html + html + "</body></html>"

    return html


In [ ]:
# Función que integra el flujo y guarda archivo para descargar
import os
from IPython.display import HTML, display

def procesar_cv_desde_interfaz_v2(archivo_pdf):
    """
    Variante que reutiliza extraer_y_clasificar_cv (tu función original),
    y usa el generador editable. Devuelve: perfil_detectado, html_string, path_del_html.
    """
    if archivo_pdf is None:
        return "Por favor, sube un archivo PDF.", "<p>No se generó HTML</p>", None

    try:
        texto_del_cv = leer_pdf(archivo_pdf.name)  # usa tu función leer_pdf existente
        if not texto_del_cv:
            return "El archivo PDF no contiene texto extraíble.", "<p>No se generó HTML</p>", None

        # 1) Extraer y clasificar (tu función existente)
        perfil_detectado, datos_extraidos = extraer_y_clasificar_cv(texto_del_cv)

        # 2) Generar HTML editable
        html_portfolio = generar_portfolio_html_editable(datos_extraidos, perfil_detectado)

        # 3) Si el LLM no puso toolbar, inyectarlo (fallback)
        html_portfolio = ensure_editable_toolbar(html_portfolio)

        # 4) Guardar archivo .html para descarga
        ruta_html = "/content/portafolio_editable_generado.html"
        with open(ruta_html, "w", encoding="utf-8") as f:
            f.write(html_portfolio)

        # 5) Retornar perfiles y ruta (Gradio puede usar la ruta en un gr.File)
        return perfil_detectado, html_portfolio, ruta_html

    except Exception as e:
        err = f"Ocurrió un error al procesar: {e}"
        return err, f"<html><body><h1>Error</h1><p>{e}</p></body></html>", None

# Opcional: interfaz Gradio separada (no sobrescribe la tuya)
import gradio as gr

interfaz_editable = gr.Interface(
    fn=procesar_cv_desde_interfaz_v2,
    inputs=gr.File(type="filepath", label="Sube tu CV en formato PDF"),
    outputs=[
        gr.Textbox(label="Clasificación del Perfil Profesional"),
        gr.HTML(label="Portafolio Web Editable (preview)"),
        gr.File(label="Descargar HTML editable")
    ],
    title="Generador de Portafolios Web EDITABLE",
    description="Versión que entrega un HTML especialmente marcado para que un frontend lo edite fácilmente."
)

# Para lanzar: interfaz_editable.launch(share=True)


In [ ]:
import gradio as gr
import os

def flujo_cv_a_portafolio(archivo_pdf):
    if archivo_pdf is None:
        return "Por favor, sube un archivo PDF.", "<p>No se generó HTML</p>", None, "{}", "<p>No se generó vista</p>"

    try:
        # 1) Leer texto desde PDF
        texto_del_cv = leer_pdf(archivo_pdf.name)
        if not texto_del_cv:
            return "El PDF no contiene texto extraíble.", "<p>No se generó HTML</p>", None, "{}", "<p>No se generó vista</p>"

        # 2) Extraer y clasificar
        perfil_detectado, datos_extraidos = extraer_y_clasificar_cv(texto_del_cv)

        # 3) Generar HTML editable
        html_portfolio = generar_portfolio_html_editable(datos_extraidos, perfil_detectado)
        html_portfolio = ensure_editable_toolbar(html_portfolio)

        # 4) Guardar archivo para descarga
        ruta_html = "/content/portafolio_editable.html"
        with open(ruta_html, "w", encoding="utf-8") as f:
            f.write(html_portfolio)

        # 5) Crear iframe embebido para mostrar la web activa
        iframe_code = f"""
        <iframe src='file://{os.path.abspath(ruta_html)}'
                width='100%' height='600'
                style='border:1px solid #ccc; border-radius:8px;'>
        </iframe>
        """

        # 6) Devolver todo
        return (
            perfil_detectado,
            html_portfolio,
            ruta_html,
            datos_extraidos.model_dump_json(indent=2),
            iframe_code
        )
    except Exception as e:
        return f"Error: {e}", "<p>No se generó HTML</p>", None, "{}", "<p>No se generó vista</p>"


interfaz_final = gr.Interface(
    fn=flujo_cv_a_portafolio,
    inputs=gr.File(type="filepath", label="Sube tu CV en formato PDF"),
    outputs=[
        gr.Textbox(label="Clasificación del Perfil Profesional"),
        gr.HTML(label="Código HTML generado (editable)"),
        gr.File(label="Descargar HTML Editable"),
        gr.Json(label="Datos del CV (JSON estructurado)"),
        gr.HTML(label="Vista interactiva del Portafolio (con botones activos)")
    ],
    title="Generador de Portafolios Web EDITABLE",
    description="Sube tu CV en PDF y genera automáticamente un portafolio web editable + datos en JSON."
)

interfaz_final.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://136c6f4658888eacb5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
